In [49]:
# -*- coding: UTF-8 -*-
import json
import os
from os.path import join as pathj
import datetime
from PIL import Image
import cv2

def ensure_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    return folder

def line2str(l):
    str1=''
    for ele in l[0:-1]:
        str1 = str1 + str(ele)+' '
    return str1 + str(l[-1])

def gen_class2id(classes):
    cls2id = {}
    for k,v in enumerate(classes):
        cls2id[v] = k
    return cls2id

def xyxy2xywh(box):
    x1 = min(box[0], box[2])
    y1 = min(box[1], box[3])
    w = abs(box[2]-box[0])
    h = abs(box[3]-box[1])
    return [x1, y1, w, h, w*h]

def gen_empty_cocojson_instance():
    # 创建 总标签data 
    now = datetime.datetime.now()
    instance = dict(
        info=dict(
            description=None,
            url=None,
            version=None,
            year=now.year,
            contributor=None,
            date_created=now.strftime("%Y-%m-%d %H:%M:%S.%f"),
        ),
        licenses=[dict(url=None, id=0, name=None,)],
        images=[
            # license, url, file_name, height, width, date_captured, id
        ],
        type="instances",
        annotations=[
            # segmentation, area, iscrowd, image_id, bbox, category_id, id
        ],
        categories=[
            # supercategory, id, name
        ],
    )

    return instance

def gen_one_data_dict(img_name, h, w, id):
    data = {
        'file_name': os.path.basename(img_name),
        'height': h,
        'width': w,
        'id': id        
    }
    return data

def gen_one_anno_dict(cat_id, img_id, bbox: list, area, anno_id):
    anno = {
        'segmentation':[],
        'area': area, #直接取绝对值
        'iscrowd': 0,
        'image_id': img_id,
        'bbox': bbox,
        'category_id': cat_id,   
        'id': anno_id

    }
    return anno
    
def gen_category_dict(class_name):
    ins = []
    for k,v in enumerate(class_name):
        one_cate = {'id':k, 'name':v,'supercategory':v}
        ins.append(one_cate) 
    return ins

def read_txt(abs_txt_name):
    with open(abs_txt_name,'r', encoding='utf-8') as f:
        for line in f.readlines():
                line = line.replace(')','').replace('(','')
                row = line.strip().split(',')
                print(row)
# 读取json
def read_json(abs_txt_name):
    with open(abs_txt_name, 'r', encoding='utf-8') as f: #读文件名
        ret_dic = json.load(f)
        return ret_dic
        
def save_json(new_name,json_ins):
    with open(new_name, 'w',encoding='utf-8') as f:
        json.dump(json_ins ,f ,indent=2,ensure_ascii=False)#zw
        print('{} saved'.format(new_name)) 

In [50]:
import numpy as np

root_dir = r'D:\sa_other\NWPU VHR-10 dataset\imgs'
file_names = []
for name in os.listdir(root_dir):
    file_names.append(name.split('.')[0])
file_names = np.array(file_names)
np.random.seed(777)
np.random.shuffle(file_names)
t_name = file_names[:585].tolist()
v_name = file_names[585:].tolist()
#打乱

In [54]:
coco_ins = gen_empty_cocojson_instance()
# print(coco_ins['images'])
# print(coco_ins['annotations'])

class_name = ['飞机', '舰船', '油罐', '棒球场','网球场','篮球场','田径场','港口','桥梁','汽车']
class_name_en = ['airplane', 'ships','oil tank', 'ball park', 'tennis court', 'basketball', 'track', 'port', 'bridge', 'car']
cate_ins = gen_category_dict(class_name_en)
coco_ins['categories'] = cate_ins
import copy
val_coco_ins = copy.deepcopy(coco_ins)
# print(coco_ins['categories'])
# os.path.basename('sdsd\sdsds\asdasd\l.ll')


In [55]:
#--- folder path：data目录
root_dir = r'D:\sa_other\NWPU VHR-10 dataset\gtwithimg'
# D:\sa_data\other\NWPU VHR-10 dataset\gtwithimg
# save_dir = r'D:\sa_other\NWPU VHR-10 dataset\coco json'#保存路径 images,labels
img = 'images'
label ='annotations'
img_part = coco_ins[img]
anno_part = coco_ins[label]

name_id = 0
anno_id = 0
anno_count =0
max_det= 0
min_det =10

for name in [x for x in os.listdir(root_dir) if x.endswith('txt')]:
    # if name.split('.')[0] not in v_name: # if name in trainning set
    #     continue
    if name.split('.')[0] not in v_name: # if name in trainning set
        continue
    file_path = os.path.join(root_dir,name)
    img_name = file_path[:-4]+'.jpg'
    txt =[]
    img = cv2.imread(img_name, flags=cv2.IMREAD_COLOR)
    # img = Image.open(img_name)#txt怎么取名的这个问题
    name_id += 1
    data_ins = gen_one_data_dict(img_name, img.shape[0], img.shape[1], name_id)
    img_part.append(data_ins) #宽高、名字
    #读取txt文件
    with open(file_path,'r',encoding='utf-8') as f:
        per_img_obj_num = 0
        for line in f.readlines():
            if  line =='':
                continue
            else:
                line = line.replace(')','').replace('(','').replace(' ','')
                row = line.strip().split(',')
                if (len(row)< 5):
                    continue
                else:
                    per_img_obj_num +=1
                    anno_count+=1
                    x1 = float(row[0])
                    y1 = float(row[1])
                    x2 = float(row[2])
                    y2 = float(row[3])
                    box =xyxy2xywh([x1,y1,x2,y2])#左下右上 coco 是[x y w h]
                    cls_id = int(row[-1])-1 # 最后一行纯数字 id从1开始
                    anno_id+=1
                    anno_ins = gen_one_anno_dict(cls_id, name_id, box[:4], box[4], anno_id)
                    anno_part.append(anno_ins) #宽高、名字
            max_det = max(per_img_obj_num, max_det)
            min_det = min(per_img_obj_num, min_det)
    # print(f'{name_id}图片共有 anno:{anno_count}')
    # abs_save_name = pathj(save_dir,save_name)

print('total transform: %s'% (anno_id))
print(f'min:{min_det}, max:{max_det}')

total transform: 425
min:1, max:31


In [56]:

save_dir = r'D:\sa_other\nv10-coco\annotations'
save_json(pathj(save_dir, "val_en.json"), coco_ins)

D:\sa_other\nv10-coco\annotations\val_en.json saved


In [53]:
save_dir = r'D:\sa_other\nv10-coco\annotations'
save_json(pathj(save_dir, "train_en.json"), coco_ins)

D:\sa_other\nv10-coco\annotations\train_en.json saved


In [39]:
import yaml

c = {'names':{}}
for k,v in enumerate(class_name_en):
    c['names'][k] = v
print(yaml.dump(c, allow_unicode=True))
with open('b1.yaml', 'w') as file:
    file.write(yaml.dump(c, allow_unicode=True))

names:
  0: airplane
  1: ships
  2: oil tank
  3: ball park
  4: tennis court
  5: basketball
  6: track
  7: port
  8: bridge
  9: car

